# IBM Applied Data Science Capstone Project by Coursera

<b>Our aim is to find the areas in the neighbourhood of Delhi, India which are best suited for opening the cafe.</b>

- Get the data of the neighbourhoods of Delhi from wikipedia
- Use the geocoder to get the coordinates of the neighbourhoods
- Use the folium to put coordinates on map
- Use foursquare API to explore the neighbourhood
- Explore the cluster using K- means 
- Choose the cluster which is best for opening the cafe in Delhi

Install the libraries needed

In [5]:
!conda install -c conda-forge geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    ratelim:         0.1.6-py_2        conda-forge

The following packages will be UPDATED:

    

In [7]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


altair-3.2.0         | 770 KB    | #################################

<b>Import the libraries</b>

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


<b>Import the data by web scraping the neighbourhoods of delhi from wikipedia</b>

In [9]:

# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Neighbourhoods_of_Delhi").text
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
# create a list to store neighborhood data
neighborhoodList = []

# append the data into the list
for row in soup.findAll("li"):
    neighborhoodList.append(row.text)
    # create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

df=kl_df[12:197].reset_index(drop=True)
df.head()

,Neighborhood
0,Adarsh Nagar
1,Ashok Vihar
2,Azadpur
3,Bawana
4,Begum Pur


Size of dataframe

In [10]:
df.shape

(185, 1)

<b>Get the coordinates of the neighbourhoods</b>

In [11]:

def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Delhi, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
cords=[]
for neighborhood in df["Neighborhood"].tolist() :
    cords.append(get_latlng(neighborhood))
cords

[[28.72034000000008, 77.17266000000006],
 [28.690420000000074, 77.17606000000006],
 [28.712420000000066, 77.17311000000007],
 [28.800590000000057, 77.03473000000008],
 [28.732600000000048, 77.05217000000005],
 [28.707083847364615, 77.20451782139037],
 [28.72876000000008, 77.17564000000004],
 [28.737310000000036, 77.03963000000005],
 [28.688010000000077, 77.15866000000005],
 [28.711690000000033, 77.20205000000004],
 [28.696560000000034, 77.13896000000005],
 [28.705010000000073, 77.18950000000007],
 [28.839770000000044, 77.07693000000006],
 [28.695890000000077, 77.13726000000008],
 [28.68580000000003, 77.13195000000007],
 [28.719440000000077, 77.10151000000008],
 [28.733520000000055, 77.10403000000008],
 [28.714230000000043, 77.15745000000004],
 [28.67557000000005, 77.16190000000006],
 [28.661060000000077, 77.20438000000007],
 [28.656240000000025, 77.23233000000005],
 [28.648560000000032, 77.23068000000006],
 [28.676690000000065, 77.21762000000007],
 [28.654567834071344, 77.2334191865485

In [12]:
df_coords = pd.DataFrame(cords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
print(df.shape)
df

(185, 3)


,Neighborhood,Latitude,Longitude
0,Adarsh Nagar,28.720340,77.172660
1,Ashok Vihar,28.690420,77.176060
2,Azadpur,28.712420,77.173110
3,Bawana,28.800590,77.034730
4,Begum Pur,28.732600,77.052170
5,Dhaka,28.707084,77.204518
6,Jahangirpuri,28.728760,77.175640
7,Karala,28.737310,77.039630
8,Keshav Puram,28.688010,77.158660
9,Kingsway Camp,28.711690,77.202050


In [13]:
df.to_csv("delhi_df.csv", index=False)

<b>Create the map of Delhi and superimpose the neighbourhoods on it</b>

In [ ]:


address = 'Delhi, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Delhi, India {}, {}.'.format(latitude, longitude))

In [16]:
map_dl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dl)  
    
map_dl

In [17]:
map_dl.save('map_dl.html')

<b>Use foursquare API to explore the neighbourhood.</b>

In [43]:
# @hidden cell
CLIENT_ID = 'Enter CLIENT_ID' # your Foursquare ID
CLIENT_SECRET = 'Enter CLIENT_SECRET' # your Foursquare Secret
VERSION = 'VERSION'

In [20]:

radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


Dataframe of the explored neighbours

In [21]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7101, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adarsh Nagar,28.72034,77.17266,Shalimar bagh club,28.711882,77.157933,Indian Restaurant
1,Adarsh Nagar,28.72034,77.17266,Ever Bake,28.706419,77.162607,Bakery
2,Adarsh Nagar,28.72034,77.17266,Arjun Pao Bhaji,28.708468,77.184963,Indian Restaurant
3,Adarsh Nagar,28.72034,77.17266,Domino's Pizza,28.706000,77.163000,Pizza Place
4,Adarsh Nagar,28.72034,77.17266,Cafe Coffee Day,28.707934,77.161478,Coffee Shop


Groupby the data with neighbourhood

In [23]:
venues_df.groupby(["Neighborhood"])['VenueName'].count()

Neighborhood
Adarsh Nagar            9
Alaknanda              50
Anand Vihar            32
Ashok Nagar             7
Ashok Vihar            25
Azadpur                24
Babarpur                4
Badarpur                4
Bali Nagar             60
Bara Hindu Rao         68
Barakhamba Road       100
Bawana                  1
Begum Pur               4
Brij Puri               4
Chanakyapuri           74
Chandni Chowk         102
Chawri Bazaar         100
Chhattarpur            32
Chittaranjan Park      90
Civil Lines            38
Connaught Place       100
Dabri, New Delhi        4
Dariba Kalan           52
Daryaganj              43
Dayanand Colony       100
Defence Colony         95
Delhi Cantonment        8
Dhaka                  25
Dhaula Kuan            80
Dilshad Garden         11
Dwarka Sub City        32
East Vinod Nagar       12
East of Kailash       100
Fateh Nagar            53
Friends Colony         15
Gandhi Nagar            4
Gautampuri              8
Geeta Colony            5

Unique category of venue available in neighbourhood

In [24]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))
# print out the list of categories
venues_df['VenueCategory'].unique()

There are 214 uniques categories.


array(['Indian Restaurant', 'Bakery', 'Pizza Place', 'Coffee Shop',
       'Park', 'Bus Station', 'Convenience Store', 'Golf Course',
       'Athletics & Sports', 'Asian Restaurant', 'Sandwich Place',
       'Snack Place', 'South Indian Restaurant', 'Department Store',
       'Fast Food Restaurant', 'BBQ Joint', 'Market', 'Dessert Shop',
       'Basketball Court', 'Train Station', 'Light Rail Station',
       'Restaurant', 'Multiplex', 'Donut Shop', 'Gym / Fitness Center',
       'Shopping Mall', 'Playground', 'Tourist Information Center', 'ATM',
       'Chinese Restaurant', 'Café', 'Food Truck', 'Korean Restaurant',
       'Indie Movie Theater', 'Ice Cream Shop', 'Lake', 'Tea Room',
       'Electronics Store', 'Diner', 'Arts & Crafts Store', 'Smoke Shop',
       'Hotel', 'Airport', 'Mobile Phone Shop', 'Breakfast Spot',
       'Theme Park', 'Clothing Store', 'Vegetarian / Vegan Restaurant',
       "Women's Store", 'Food & Drink Shop', 'Motel', 'Grocery Store',
       'Miscellaneous Sh

Checking if cafe is available as category in the data

In [25]:
"Café" in venues_df['VenueCategory'].unique()

True

<b>Analysis of the data</b>

In [26]:
dl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dl_onehot.columns[-1]] + list(dl_onehot.columns[:-1])
dl_onehot = dl_onehot[fixed_columns]

print(dl_onehot.shape)
dl_onehot.head()

(7101, 215)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,High School,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Northeast Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Grop the data by neighbourhood and take the mean of the frequency the occourence of venue

In [27]:
dl_grouped = dl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(dl_grouped.shape)
dl_grouped

(172, 215)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,High School,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Northeast Indian Restaurant,Office,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Udupi Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Adarsh Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.111111,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.222222,0.000,0.000000,0.000000,0.000000,0.000000,0

Number of area having the cafe

In [29]:
len(dl_grouped[dl_grouped["Café"] > 0])

118

In [30]:
dl_cafe = dl_grouped[["Neighborhoods","Café"]]
dl_cafe.head()

,Neighborhoods,Café
0,Adarsh Nagar,0.0000
1,Alaknanda,0.0400
2,Anand Vihar,0.0625
3,Ashok Nagar,0.0000
4,Ashok Vihar,0.0000


<b>Applying the K Means clustering </b>

In [31]:
kclusters = 3

dl_clustering = dl_cafe.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 2, 1, 1, 1, 1, 1, 2, 1], dtype=int32)

In [32]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
dl_merged = dl_cafe.copy()

# add clustering labels
dl_merged["Cluster Labels"] = kmeans.labels_

Data gets segmented in three parts

In [33]:
dl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
dl_merged.head()

,Neighborhood,Café,Cluster Labels
0,Adarsh Nagar,0.0000,1
1,Alaknanda,0.0400,1
2,Anand Vihar,0.0625,2
3,Ashok Nagar,0.0000,1
4,Ashok Vihar,0.0000,1


In [34]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dl_merged = dl_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(dl_merged.shape)
dl_merged.head() # check the last columns!

(182, 5)


,Neighborhood,Café,Cluster Labels,Latitude,Longitude
0,Adarsh Nagar,0.0000,1,28.72034,77.17266
1,Alaknanda,0.0400,1,28.52568,77.25324
2,Anand Vihar,0.0625,2,28.65356,77.31373
3,Ashok Nagar,0.0000,1,28.69223,77.30127
4,Ashok Vihar,0.0000,1,28.69042,77.17606


Clustered data

In [35]:
print(dl_merged.shape)
dl_merged.sort_values(["Cluster Labels"], inplace=True)
dl_merged

(182, 5)


,Neighborhood,Café,Cluster Labels,Latitude,Longitude
140,Sarita Vihar,0.184211,0,28.550320,77.283430
28,Dhaula Kuan,0.150000,0,28.594895,77.167263
28,Dhaula Kuan,0.150000,0,28.594895,77.167263
106,Okhla,0.210526,0,28.532490,77.278380
21,"Dabri, New Delhi",0.250000,0,28.607590,77.087210
133,Sagar Pur,0.200000,0,28.605900,77.095540
31,East Vinod Nagar,0.250000,0,28.622580,77.306610
51,Jahangirpuri,0.250000,0,28.728760,77.175640
113,Patparganj,0.173913,0,28.629480,77.307660
162,Uttam Nagar,0.166667,0,28.617370,77.052860


<b>Map showing different cluster with different colors</b>

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dl_merged['Latitude'], dl_merged['Longitude'], dl_merged['Neighborhood'], dl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [38]:
dl_merged.loc[dl_merged['Cluster Labels'] == 0]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
140,Sarita Vihar,0.184211,0,28.550320,77.283430
28,Dhaula Kuan,0.150000,0,28.594895,77.167263
28,Dhaula Kuan,0.150000,0,28.594895,77.167263
106,Okhla,0.210526,0,28.532490,77.278380
21,"Dabri, New Delhi",0.250000,0,28.607590,77.087210
133,Sagar Pur,0.200000,0,28.605900,77.095540
31,East Vinod Nagar,0.250000,0,28.622580,77.306610
51,Jahangirpuri,0.250000,0,28.728760,77.175640
113,Patparganj,0.173913,0,28.629480,77.307660
162,Uttam Nagar,0.166667,0,28.617370,77.052860


In [39]:
dl_merged.loc[dl_merged['Cluster Labels'] == 1]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
82,Mahipalpur,0.031746,1,28.548430,77.136360
111,Paschim Vihar,0.000000,1,28.669330,77.091730
110,Partap Nagar,0.034483,1,28.669750,77.195740
68,Khanpur,0.000000,1,28.509590,77.230990
72,Kohat Enclave,0.030303,1,28.696560,77.138960
109,Pandav Nagar,0.000000,1,28.614580,77.275740
108,Palam,0.000000,1,28.591080,77.091190
105,Nizamuddin West,0.046512,1,28.589730,77.245220
104,New Usmanpur,0.000000,1,28.682490,77.256500
102,Netaji Nagar,0.035714,1,28.577470,77.185160


In [40]:
dl_merged.loc[dl_merged['Cluster Labels'] == 2]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
16,Chawri Bazaar,0.060000,2,28.648560,77.230680
130,Sadar Bazaar,0.111111,2,28.590290,77.120150
130,Sadar Bazaar,0.111111,2,28.590290,77.120150
138,Sarai Kale Khan,0.117647,2,28.584860,77.263560
132,Safdarjung Enclave,0.070000,2,28.564750,77.194630
20,Connaught Place,0.080000,2,28.633920,77.219780
17,Chhattarpur,0.093750,2,28.504560,77.173530
18,Chittaranjan Park,0.055556,2,28.538410,77.248310
134,Sainik Farm,0.095238,2,28.505500,77.214190
19,Civil Lines,0.052632,2,28.676690,77.217620


<b>Observation:</b>
From the above results we can say that the cluser 0 has the highest number of cafe in the locality and cluster 2 has moderate number of cafe. So the cluster 1 has no or very few cafe in the locality hence area that are in cluster 1 are great places for opening new cafe in Delhi. So builders, youngsters or any businessman could florish their cafe business in the area of cluser 1 because of the least competition. Opening cafe in cluser 0 would lead to oversupply and high competition. People who are having some unique ideas of cafe can even open cafe in area of cluser 2.